In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
import os, sys
import pandas as pd

In [ ]:
from ln_utils import *

# Parameters

In [ ]:
from datawand.parametrization import ParamHelper
ph = ParamHelper("../", "LNGraph", sys.argv)

In [ ]:
weight = ph.get("weight")
weight_cols = [weight] if weight != None else None

In [ ]:
data_dir = ph.get("data_dir")
output_dir = "%s/centrality_scores/" % data_dir
print(output_dir)

# Load temporal data

edges = pd.read_csv("../LNdata/edge_last_updates.csv")

In [ ]:
edges = pd.read_csv("%s/directed_temporal_edges.csv" % data_dir)

In [ ]:
edges.shape

In [ ]:
edges.head()

# Analyzing graph snapshots

### Snapshots with the number of edges

In [ ]:
snapshot_graphs, _ = get_snapshots(edges, weight_cols)

### Calculate centralities for snapshots

# TODO: several betweeness version:
- [current flow](https://networkx.github.io/documentation/networkx-2.2/reference/algorithms/generated/networkx.algorithms.centrality.current_flow_betweenness_centrality.html#networkx.algorithms.centrality.current_flow_betweenness_centrality)
- [communicability](https://networkx.github.io/documentation/networkx-2.2/reference/algorithms/generated/networkx.algorithms.centrality.communicability_betweenness_centrality.html#networkx.algorithms.centrality.communicability_betweenness_centrality)
- [percolation centrality](https://networkx.github.io/documentation/networkx-2.2/reference/algorithms/generated/networkx.algorithms.centrality.percolation_centrality.html#networkx.algorithms.centrality.percolation_centrality)

TMP = nx.Graph()
TMP.add_edges_from(snaphots[0])

%%time
comm = nx.communicability_betweenness_centrality(TMP)

%%time
betw = nx.betweenness_centrality(TMP)

%%time
betw = nx.betweenness_centrality(TMP)

In [ ]:
stats = get_snapshot_properties(snapshot_graphs, weight=weight)

In [ ]:
for idx, snap_stat in enumerate(stats):
    snap_stat.to_csv("%s/scores_%s_%i.csv" % (output_dir,weight,idx), index=False)

# Correlation time series

### Correlation of centrality measures on adjacent weeks

adj_results = dict([(corr, get_corr_sequence(stats, corr, adjacent=True)) for corr in ["pearson","spearman","kendall","w_kendall"]])

for cent in  ["deg","wdeg","pr","betw","harm"]:
    show_corr_results(adj_results, cent)

### Correlations of first week and current week centrality measures

first_results = dict([(corr, get_corr_sequence(stats, corr, adjacent=False)) for corr in ["pearson","spearman","kendall","w_kendall"]])

for cent in  ["deg","wdeg","pr","betw","harm"]:
    show_corr_results(first_results, cent)